In [ ]:
import altair as alt
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import squarify

In [ ]:
alt.renderers.enable('notebook')

In [ ]:
import wrangler
import utils

In [ ]:
receipt_filename = '01._Receipt_-_DDOReceipt_Head_Date_and_Challan_Wise.csv'
receipt_filepath = utils.get_munged_filepath(receipt_filename)
df = pd.read_csv(receipt_filepath, parse_dates=True)
receipt_df = wrangler.wrangle_data_for_receipt(df, ['RECEIPTHEAD'])

expenditure_filename = '10._Expenditure_-_DDO_Head_of_AccountSOE_and_VoucherBillNO_wise.csv'
expenditure_filepath = utils.get_munged_filepath(expenditure_filename)
df = pd.read_csv(expenditure_filepath, parse_dates=True)
expenditure_df = wrangler.wrangle_data_for_consolidated_query(df, ['DDODESC', 'DISTRICT', 'TREASURY', 'DDO'])

In [ ]:
def plot_receipt_exp_timeline():
    receipt_sample = receipt_df.resample('M')['Net Receipt'].sum()
    expenditure_sample = expenditure_df.resample('M')['NETPAYMENT'].sum()
    data = pd.concat([receipt_sample, expenditure_sample], axis=1)
    data.index.name = 'Date'
    
    data = pd.melt(data.reset_index(), id_vars='Date', value_vars=['Net Receipt', 'NETPAYMENT'],
        var_name='Category', value_name='Amount')
    
    return alt.Chart(
        data.reset_index(),
        width=700,
        height=350
        
    ).mark_line(point=True).encode(
        
        x=alt.X('Date:T',
                axis=alt.Axis(title='Timeline (monthly)', format='%b %Y')),
        y=alt.Y('Amount:Q',
                axis=alt.Axis(title='Amount', format='.2s')),
        color='Category',
        tooltip=['Category:O', alt.Text('Amount:Q', format='.3s')]
    )

In [ ]:
def compare_district_receipt_exp():
    receipt_sample = receipt_df.groupby('District')['Net Receipt'].sum()

    exp_sample = expenditure_df.groupby('DISTRICT')['NETPAYMENT'].sum()

    data = pd.concat([receipt_sample, exp_sample], axis=1).dropna()
    data.index.name = 'District'

    data = pd.melt(data.reset_index(), id_vars='District', value_vars=['Net Receipt', 'NETPAYMENT'],
            var_name='Category', value_name='Amount')
    
    base = alt.Chart(data)

    title = alt.Axis(title='Amount (in Rupees)', format='.2s')
    color_scale = alt.Scale(domain=['Net Receipt', 'NETPAYMENT'],
                            range=['#1f77b4', '#e377c2'])

    left = base.transform_filter(
        alt.expr.datum.Category == 'Net Receipt'
    ).encode(
        y=alt.X('District:N',
                axis=None),
        x=alt.X('Amount:Q', axis=title, sort=alt.SortOrder('descending')),
        color=alt.Color('Category:O', legend=None),
        tooltip=['District:N', alt.Text('Amount:Q', format='.3s')],
    ).mark_bar().properties(title='Net Receipt')

    middle = base.encode(
        y=alt.X('District:N', axis=None),
        text=alt.Text('District:N'),
    ).mark_text().properties(width=85)

    right = base.transform_filter(
        alt.expr.datum.Category == 'NETPAYMENT'
    ).encode(
        y=alt.X('District:N',
                axis=None),
        x=alt.X('Amount:Q', axis=title),
        color=alt.Color('Category:O', legend=None),
        tooltip=['District:N', alt.Text('Amount:Q', format='.3s')]
    ).mark_bar().properties(title='NETPAYMENT')

    return left | middle | right